In [1]:
import os
import json
import itertools
# from tqdm import tqdm
# from tensorflow import keras
# from IPython.display import clear_output

# from pattern.text.en import tokenize
# from gensim.models import Word2Vec

In [2]:
# Load the dictionary with the lookup results for each cell value in the tabular data
def load_json(data_json):
    with open(data_json) as json_file:
        return json.load(json_file)

In [3]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint
import requests
import xml.etree.ElementTree as ET

In [4]:
def dbo_sparql_results(query_string):
    
    classes = list([])
    dbo_prefix = 'http://dbpedia.org/ontology/'
    
    
    sparql = SPARQLWrapper('https://dbpedia.org/sparql')
    sparql.setQuery(query_string)
    
    try:
        sparql.setReturnFormat(JSON)
        qres = sparql.query().convert()
        for entity_class in qres['results']['bindings']:
            if dbo_prefix in entity_class[list(qres['results']['bindings'][0].keys())[0]]['value']:
                candicate_class = entity_class[list(qres['results']['bindings'][0].keys())[0]]['value'].split(dbo_prefix)[1]
                classes.append(candicate_class)
    except:
        pass
    
    return classes

def get_dbo_subclass(superClass):
    
    query_string = f'''
    SELECT distinct ?subClass 
    WHERE {{ ?subClass rdfs:subClassOf dbo:{superClass}. }}
    '''
    return dbo_sparql_results(query_string)


def get_dbo_superclass(subclass):
    
    query_string = f'''
    SELECT distinct ?superClass 
    WHERE {{ dbo:{subclass} rdfs:subClassOf ?superClass . }}
    '''
    
    return dbo_sparql_results(query_string)


def get_dbo_superclasses(subclass):
    classes = list([])
    
    try:
        parent = get_dbo_superclass(subclass)
    except:
        return []
    
    while len(parent) > 0:
        classes.append(parent[0])
        parent = get_dbo_superclass(parent[0])
    return classes

get_dbo_superclasses('Actor')

['Artist', 'Person', 'Animal', 'Eukaryote', 'Species']

In [5]:
output_folder = 'output\\2019_round1\\'
cnn_model_directory = os.getcwd()+'\\output\\2019_round1\\cnn_models'

dict_predictions_top5_sliding = load_json(output_folder+'dict_predictions_top5_sliding.json')
data = load_json(output_folder+'data.json')

In [ ]:
# print(dict_predictions_top5_sliding['8468806_0_4382447409703007384']['1']['gt'])
found_strict = 0
found_relaxed = 0
total = 0
for filename in dict(itertools.islice(dict_predictions_top5_sliding.items(), 1000)):
    index = 0
    for col in dict_predictions_top5_sliding[filename]:
        actual_cls = dict_predictions_top5_sliding[filename][col]['gt']
        top1_cls = dict_predictions_top5_sliding[filename][col]['pred'][0][0]
        if top1_cls == 'Agent':
            try:
                top1_cls = dict_predictions_top5_sliding[filename][col]['pred'][1][0]
            except: pass
            
        offspring_cls = get_dbo_subclass(top1_cls)    
        
        if top1_cls == actual_cls:
            found_strict += 1
            found_relaxed +=1
        elif top1_cls in get_dbo_superclasses(actual_cls):
            found_relaxed += 0.5
        total += 1
        
        
print(f"Found Strict: {found_strict*100/total}")   
print(f"Found Relaxed: {found_relaxed*100/total}")     


# Found Strict: 46.2962962962963
# Found Relaxed: 62.5

In [16]:
# print(dict_predictions_top5_sliding['8468806_0_4382447409703007384']['1']['gt'])
found_strict = 0
found_relaxed = 0
total = 0
for filename in dict(itertools.islice(dict_predictions_top5_sliding.items(), 2)):
    index = 0
    print(f"-----------------------------------------------")
    print(f"Filename: {filename}")
    for col in dict_predictions_top5_sliding[filename]:
        actual_cls = dict_predictions_top5_sliding[filename][col]['gt']
        top1_cls = dict_predictions_top5_sliding[filename][col]['pred'][0][0]
        if top1_cls == 'Agent':
            try:
                top1_cls = dict_predictions_top5_sliding[filename][col]['pred'][1][0]
            except: pass
            
        offspring_cls = get_dbo_subclass(top1_cls)
        print(f"Column: {col},Actual class: {actual_cls}, Predicted Class: {top1_cls}, Column Title:")
        print(f"Offspring: {get_dbo_subclass(top1_cls)}")
        
        


-----------------------------------------------
Filename: 58891288_0_1117541047012405958
Column: 1,Actual class: Film, Predicted Class: Work, Column Title:
Offspring: ['Software', 'Artwork', 'Film', 'RadioProgram', 'TelevisionEpisode', 'TelevisionSeason', 'TelevisionShow', 'Website', 'WrittenWork', 'Document', 'MusicalWork', 'Database', 'CollectionOfValuables', 'LineOfFashion', 'قیمتی_اشیاء_کا_مجموعہ۔']
Column: 3,Actual class: Person, Predicted Class: Person, Column Title:
Offspring: ['MilitaryPerson', 'Artist', 'Athlete', 'Cleric', 'Criminal', 'Philosopher', 'Politician', 'Religious', 'Royalty', 'Writer', 'OrganisationMember', 'Presenter', 'Architect', 'Astronaut', 'BeautyQueen', 'BusinessPerson', 'Chef', 'Economist', 'Engineer', 'HorseTrainer', 'Journalist', 'Model', 'Noble', 'Scientist', 'Youtuber', 'Judge', 'PlayboyPlaymate', 'Coach', 'SportsManager', 'Monarch', 'OfficeHolder', 'Academic', 'AmericanLeader', 'Archeologist', 'Aristocrat', 'DBpedian', 'Egyptologist', 'Farmer', 'Lawyer

In [65]:
temp = dict_predictions_top5_sliding['8468806_0_4382447409703007384']['2']['pred']
print(get_dbo_subclass('PopulatedPlace'))
temp = [t for t in temp if t[1]>10]
print(temp)

temp = [t for t in temp if t[0] in get_dbo_subclass('PopulatedPlace')]
print(temp)

print(len(temp)/len(get_dbo_subclass('PopulatedPlace')))

['Continent', 'Settlement', 'Region', 'Country', 'Island', 'Agglomeration', 'Community', 'GatedCommunity', 'Intercommunality', 'Locality', 'State', 'Street', 'Territory']
[['PopulatedPlace', 100.0], ['Settlement', 78.57], ['Location', 75.0], ['Country', 71.43], ['Place', 64.29]]
[['Settlement', 78.57], ['Country', 71.43]]
0.15384615384615385


In [68]:
print(get_dbo_subclass('Country'))

['HistoricalCountry']


In [44]:
# cell_values = load_json(output_folder+'cell_values.json')
# cell_values.pop('parsed_files')

cls_list = list()
for cell in dict(itertools.islice(cell_values.items(), 200)):
    for entity in cell_values[cell]['candidate_entities']:
        cls_list.append(cell_values[cell]['candidate_entities'][entity]['candidate_classes'])

In [48]:
len(cls_list)

758

In [62]:
final_list = list()

# for cls_combo in set(tuple(row) for row in cls_list):
for cls_combo in itertools.islice(set(tuple(row) for row in cls_list), 5):
    print(cls_combo)
    for cls in cls_combo:
        pass


('MusicGenre', 'Agent')
('Person', 'Artist', 'Actor', 'Agent')
('Person', 'Royalty', 'Agent')
('Crater',)
('Station', 'Place', 'ArchitecturalStructure', 'Location', 'Infrastructure')
